In [ ]:

# coding: utf-8

# In[ ]:


import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import pandas as pd
import os
import requests
from gevent.pywsgi import WSGIServer
import exchanges
import historicaldata as hisdata
from multiprocessing.pool import ThreadPool
import pkgutil

exchange_list = []
prefix = exchanges.__name__ + "."
for importer, modname, ispkg in pkgutil.iter_modules(exchanges.__path__,prefix):
    print ("Found submodule %s (is a package: %s)"% (modname, ispkg))
    module = __import__(modname, fromlist="exchanges")
    exchange_list.append(module)
    

symbols = {'BTC':'Bitcoin','ETH':'Ethereum','BCH':'Bitcoin Cash','LTC':'Litecoin',
           'EOS':'EOS','XRP':'Ripple','STEEM':'Steem','BSV':'Bitcoin SV','TRX':'TRON',
           'ADA':'Cardano','MIOTA':'IOTA','XMR':'Monero','BNB':'Binance Coin',
           'DASH':'Dash','XEM':'NEM','ETC':'Ethereum Classic','NEO':'NEO','WAVES':'Waves','ZEC':'Zcash'}

tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

def getRealtimeData(symbol):
    results = []
    pool = ThreadPool()
    for exchange in exchange_list:
        print(exchange)
        results.append(pool.apply_async(exchange.getUSDPrice, args = (symbol,)))

    price_list = [r.get() for r in results if r is not None and r.get() is not None]
    pool.close()
    pool.join()

    if not price_list:
        return None
    
    df = pd.DataFrame(price_list)
    print(df)
    df = df[['source','bid','ask','trade']]
    df.columns=['Exchange','BestBid','BestAsk','Last Trade Price']
    return df

    
def getHistoricalData(symbol,interval):
    data_list = hisdata.getHistoricalData(symbol,interval)
    return pd.DataFrame(data_list)
    
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Cryptocurrency Exchanges Depth'

app.layout = html.Div(
[  

        html.Div(
        className="app-header",
        children=[
            html.Div('Cryptocurrency Exchanges Depth', className="app-header--title")
        ]
        ),

        html.Div(
        [
                html.Label('Symbol'),
                dcc.Dropdown(
                    id='symbol',
                    options=[
                        {'label': symbol+' '+name, 'value': symbol} for symbol,name in symbols.items()
                    ],value='BTC')
        ],style={'width': '20%', 'display': 'inline-block'}),
        
        html.Div(
        [
            html.Div(id='my-table',style={'width': '40%', 'display': 'inline-block'}),
            html.Div(
                [
                    dcc.Tabs(id="tabs-history", value='tab-history', children=[
                    dcc.Tab(label='24 hours trend', value='tab-24-hours',style=tab_style, selected_style=tab_selected_style),
                    dcc.Tab(label='1 month trend', value='tab-1-month',style=tab_style, selected_style=tab_selected_style),
                    dcc.Tab(label='1 year trend', value='tab-1-year',style=tab_style, selected_style=tab_selected_style),
                    ],style=tabs_styles),
                    dcc.Graph(id='trend-chart')
                ],style={'width': '40%', 'display': 'inline-block','float':'right'})
    
        ]),
        dcc.Interval(
                    id='interval-component',
                    interval=10*1000, # in milliseconds
                    n_intervals=0
                ),
        html.Div (  
               className="footer",
               children=[
                html.Div([
                    html.P('Biglion.com.au'),
                    html.P('2018 copyright'),
                ], style={'width': '49%', 'display': 'table-cell','vertical-align': 'middle'}),
                html.Div([
                    html.P('Donation'),
                    html.P('XBT： 3NCc5DXMBzSnaZgc97E6MKtwUf52HgFArK        '),
                    html.P('ETH： 0x0058501228fa553aa1aba12baf9f7c46b3822fef'),
                    html.P('LTC： MGDVgCeMRFWkDYyAki3Mm3WX4Pdp7D3kbe        ')
                ], style= {'width': '49%', 'display': 'table-cell'})
               ]
        ),
]

)

@app.callback(Output('my-table', 'children'), [Input('symbol', 'value'),Input('interval-component', 'n_intervals')])
def table_update(selected_dropdown_value,n):
    df1 = getRealtimeData(selected_dropdown_value)
    return generate_table(df1)

@app.callback(Output('trend-chart', 'figure'),
              [Input('tabs-history', 'value'),Input('symbol', 'value')])
def graph_update(tab,selected_dropdown_value):
    df = None
    if tab == 'tab-24-hours':
        df = getHistoricalData(selected_dropdown_value,'day')
    elif tab == 'tab-1-month':
        df = getHistoricalData(selected_dropdown_value,'month')
    elif tab == 'tab-1-year':
        df = getHistoricalData(selected_dropdown_value,'year')
    if df is not None:
        return {'data': [go.Scatter(x=df['time'],
                                    y=df['close'],
                                    mode='lines+markers')
                            ]
                }
    else:
        return
    
if __name__ == '__main__':
    app.run_server(debug=False,port=8000)

# if __name__ == '__main__':
#     http_server = WSGIServer(('', 5000), app)
#     http_server.serve_forever()




Found submodule exchanges.BTCMarkets (is a package: False)
Found submodule exchanges.OKEx (is a package: False)
Found submodule exchanges.binance (is a package: False)
Found submodule exchanges.bitfinex (is a package: False)
Found submodule exchanges.bittrex (is a package: False)
Found submodule exchanges.coinbene (is a package: False)
Found submodule exchanges.gateio (is a package: False)
Found submodule exchanges.independentreserve (is a package: False)
Found submodule exchanges.okcoin (is a package: False)
Found submodule exchanges.poloniex (is a package: False)
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jan/2019 22:18:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2019 22:18:30] "GET /assets/header.css?m=1546132873.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2019 22:18:30] "GET /assets/gtag.js?m=1546595461.5269017 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2019 22:18:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2019 22:18:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2019 22:18:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2019 22:18:32] "GET /assets/blockchain_featured.jpeg HTTP/1.1" 200 -


<module 'exchanges.BTCMarkets' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\BTCMarkets.py'>
<module 'exchanges.OKEx' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\OKEx.py'>
<module 'exchanges.binance' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\binance.py'>
<module 'exchanges.bitfinex' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\bitfinex.py'>
<module 'exchanges.bittrex' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\bittrex.py'>
<module 'exchanges.coinbene' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\coinbene.py'>
<module 'exchanges.gateio' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\gateio.py'>
<module 'exchanges.independentreserve' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\independentreserve.py'>
<module 'exchanges.okcoin' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\okcoin.py'>
<module 'exchanges.poloniex' from 'C:\\Users\\Administrator\\cryptode

127.0.0.1 - - [05/Jan/2019 22:18:37] "POST /_dash-update-component HTTP/1.1" 200 -


             ask            bid              source          trade
0         3780.7        3773.36          BTCMarkets        3780.69
1      3796.7747      3795.7214                okex      3795.7215
2        3797.92        3796.63             binance        3797.92
3        3833.25        3825.65             bittrex        3822.83
4         3800.6        3797.27            coinbene        3797.28
5        3801.05        3800.14           gateio.io        3800.14
6        3873.21        3820.01  independentreserve        3810.23
7        3832.66        3830.47              OKcoin        3832.69
8  3794.87871485  3792.45161489            poloniex  3794.87871416


...

3796.28
{'source': 'coinbene', 'bid': 39.1662, 'ask': 63.9332, 'trade': 49.2643}
{'source': 'poloniex', 'bid': '49.05243905', 'ask': '49.85927245', 'trade': '49.29080000'}
           ask          bid     source        trade
0      48.4836      48.2199       okex      48.4836
1      49.2634      49.2444    binance      49.2671
2      63.9332      39.1662   coinbene      49.2643
3        49.49        48.98  gateio.io        48.98
4  49.85927245  49.05243905   poloniex  49.29080000
<module 'exchanges.BTCMarkets' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\BTCMarkets.py'>
<module 'exchanges.OKEx' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\OKEx.py'>
<module 'exchanges.binance' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\binance.py'>
<module 'exchanges.bitfinex' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\bitfinex.py'>
<module 'exchanges.bittrex' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\bittrex.py'>
<module '

127.0.0.1 - - [05/Jan/2019 22:28:05] "POST /_dash-update-component HTTP/1.1" 200 -


{'source': 'poloniex', 'bid': '49.05243905', 'ask': '49.85917997', 'trade': '49.29080000'}
           ask          bid     source        trade
0      48.4836      48.2199       okex      48.4836
1      49.2468      49.2317    binance      49.2317
2      63.9303      39.1683   coinbene      49.2621
3        49.49        48.98  gateio.io        48.98
4  49.85917997  49.05243905   poloniex  49.29080000
<module 'exchanges.BTCMarkets' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\BTCMarkets.py'>
<module 'exchanges.OKEx' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\OKEx.py'>
<module 'exchanges.binance' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\binance.py'>
<module 'exchanges.bitfinex' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\bitfinex.py'>
<module 'exchanges.bittrex' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\bittrex.py'>
<module 'exchanges.coinbene' from 'C:\\Users\\Administrator\\cryptodepth_new\\exchanges\\c

127.0.0.1 - - [05/Jan/2019 22:28:15] "POST /_dash-update-component HTTP/1.1" 200 -


{'source': 'poloniex', 'bid': '49.05243905', 'ask': '49.85917995', 'trade': '49.29080000'}
           ask          bid     source        trade
0      48.4836      48.2199       okex      48.4836
1      49.2466      49.2238    binance      49.2238
2       63.923      39.1638   coinbene      49.2565
3        49.49        48.98  gateio.io        48.98
4  49.85917995  49.05243905   poloniex  49.29080000
